In [2]:
# pip install langchain

In [3]:
# %pip install --upgrade langchain-together

In [4]:
from langchain_together import ChatTogether
from langchain.schema import HumanMessage
import json
from langchain.agents import Tool, initialize_agent, AgentType


In [5]:
api_key = 'f3b01991e5fcf24a0e024c67791f79a4f676aa55188df99925dafe784e82c9cf'
llm = ChatTogether(
    together_api_key=api_key,
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"  # hoặc bất kỳ model chat nào bạn chọn
)


In [6]:
def schedule_room_booking(data):
    print(data)
    return 'Book successfully'
def check_room_available(data):
    print(data)
    return '''
        room_id: 123
    '''

In [7]:
schedule_room_booking_prompt = '''
This function use to booking room.
Input with json format like:
{{
    room_id: 'room_id_value',
    from: 'dd/mm/yyyy'
    to: 'dd/mm/yyyy'
}}
Output is message notify if room is booked
'''
check_room_available_prompt = '''
This function use to check when and what room is available.
Input with json format like
{{
    from: 'dd/mm/yyyy',
    to: 'dd/mm/yyyy'
    
}}
ouput with json format like:
{{
    room_id: 'room_id_value'
}}
'''

In [8]:

tools = [
    Tool(
        name="ScheduleRoomBooking",
        func=schedule_room_booking,
        description= schedule_room_booking_prompt 
    ),
    Tool(
        name="CheckRoom",
        func= check_room_available,
        description= check_room_available_prompt
        ),

]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=5
)

C:\Users\admin\AppData\Local\Temp\ipykernel_18780\3015613068.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
prompt = f'''
You are an assistant to help book room, check if room avalable. Conversation below is your chat with user.
    User: I want to reserve a room today
    Bot: 
'''

In [10]:
agent.invoke(prompt)



> Entering new AgentExecutor chain...
To assist the user in reserving a room for today, I first need to check which rooms are available for today. This involves using the CheckRoom function to see what rooms can be booked for the current date.

Action: CheckRoom
Action Input: {
    from: 'current_date',
    to: 'current_date'
}{
    from: 'current_date',
    to: 'current_date'
}

Observation: 
        room_id: 123
    
Thought:Since the CheckRoom function has provided a room_id (123) that is available for today, I can now assist the user in booking this room. To confirm the booking, I will use the ScheduleRoomBooking function with the available room_id and the current date as the from and to dates.

Action: ScheduleRoomBooking
Action Input: {
    room_id: '123',
    from: 'current_date',
    to: 'current_date'
}{
    room_id: '123',
    from: 'current_date',
    to: 'current_date'
}

Observation: Book successfully
Thought:I now know the final answer
Final Answer: Your room with id 12

{'input': '\nYou are an assistant to help book room, check if room avalable. Conversation below is your chat with user.\n    User: I want to reserve a room today\n    Bot: \n',
 'output': 'Your room with id 123 has been booked successfully for today.'}

In [4]:
import requests

def get_all_rooms(jwt_token: str):
    url = "http://localhost:8080/rooms/all"  # Đổi URL này thành URL thật của bạn

    headers = {
        "Authorization": f"Bearer {jwt_token}",
        # "Accept": "application/json"
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # raise nếu HTTP status >= 400
        return response.json()  # trả về dữ liệu JSON từ server
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Connection Error:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("Other Error:", err)

    return None  # Nếu có lỗi
if __name__ == "__main__":
    # Token thật từ Redis hoặc Keycloak
    token = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGeldUN3dja0U5dzVraWRmV180Q0ZLRnB4NlFEV2dkdS1uY0hvcUNRZTBVIn0.eyJleHAiOjE3NTM0MDkwMTcsImlhdCI6MTc1MzM3MzAxNywianRpIjoib25ydHJvOjE4Njk5Y2Q2LWE1YWMtNGViYS0yZWY4LTJjYjc4MGE1YjM3MCIsImlzcyI6Imh0dHA6Ly9sb2NhbGhvc3Q6ODA4MS9yZWFsbXMvVXNlclNjaGVkdWxlIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjNjM2FlMWM2LTdjOTItNDFmMi1hMjY5LTM4YmY4ZThlMmZkZiIsInR5cCI6IkJlYXJlciIsImF6cCI6InVzZXJfc2NoZWR1bGUiLCJzaWQiOiIyM2Y5NjcwZC02MmI0LTQ3NGQtYmU0Zi02ZmFiMzIxNzk3YzkiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbIioiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiTUFOQUdFUiIsInVtYV9hdXRob3JpemF0aW9uIiwiQURNSU4iLCJVU0VSIiwiZGVmYXVsdC1yb2xlcy11c2Vyc2NoZWR1bGUiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6Im9wZW5pZCBlbWFpbCBwcm9maWxlIiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJuYW1lIjoiYWRtaW4gYWRtaW4iLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJhZG1pbjEiLCJnaXZlbl9uYW1lIjoiYWRtaW4iLCJmYW1pbHlfbmFtZSI6ImFkbWluIiwiZW1haWwiOiJhYmNAZ21haWwuY29tIn0.NHUr3CYtz3h9aR7uzVpgWYe-o3l2gTAn2PKjWuM4qFiOL6jX00ck_zM5_p7c_yZZRqb23_YnagkWx_QPKkcH2e-xKJdmiQUaz1pwvLc-0f9YDWr5J83VOv-sdF03rkGOjLjMTNkYqfDHWgcOTGjRqgI7O2AxjudrdKOdPh0CsbTp2-hwIqUQ_UCpmz4P2rnde5gEIhk_eglFwmZ5-UWpH2B0ZUEvtEBkLeGDGEXVbX7IwsNcUAY5jelD1eYxLkAjNz6Padxza821K_oYQSzDeIF-iXOPjsRrIdwTvgvXc-E5he3neyHOO-oRAT2JvFEbZqYqO4u91msdmuqT1YNXxQ"  # cắt ngắn

    result = get_all_rooms(token)
    if result:
        print("Danh sách phòng:")
        print(result)
    else:
        print("Không thể lấy danh sách phòng.")


Danh sách phòng:
{'statusCode': 1000, 'message': 'Get all rooms successfully', 'data': [{'name': '2B21', 'location': 'Qu?n 9', 'capacity': 50}]}
